In [ ]:
import simulate
import withdrawal
import market
import harvesting
import metrics
from decimal import Decimal as D
import plot

import pandas
import numpy

In [ ]:
class CappedVPW(withdrawal.VPW):
    ceiling = 80000

    def _calc(self):
        amt = super()._calc()
        ceiling = self.ceiling * self.cumulative_inflation
        return min(amt, ceiling)

In [ ]:
class SplitVPW(withdrawal.VPW):
    def __init__(self, portfolio, harvest_strategy, years_left=35):
        super().__init__(portfolio, harvest_strategy, years_left=years_left)
        self.initial_floor = D('.03') * D('.6') * self.portfolio.value
        
    def _calc(self):
        floor = self.initial_floor * self.cumulative_inflation
        rest_of_portfolio = self.portfolio.value * D('.4')
        
        rate = (self.portfolio.stocks_pct * self.stock_growth_rate
                + self.portfolio.bonds_pct * self.bond_growth_rate)

        amt = metrics.pmt(rate, self.years_left, rest_of_portfolio)        
        # max out at 20% of the current portfolio...this allows it to run
        # "indefinitely"
        vpw_amount = min(amt, rest_of_portfolio / 5)
        
        return floor + vpw_amount

In [ ]:
def run_one(withdrawal_strategy=withdrawal.VPW):
    MARKET = market.Returns_US_1871()

    years = 30
    end_year = 2018 - years + 1
    series = pandas.Series(index=numpy.arange(MARKET.start_year, end_year))

    for start in range(MARKET.start_year, end_year):
        annual_data = simulate.withdrawals(MARKET.iter_from(start),
                                           years=years,
                                           harvesting=harvesting.N_60_RebalanceHarvesting,
                                           withdraw=withdrawal_strategy)
        min_wd = annual_data[0].withdraw_r
        for y in annual_data:
            min_wd = min(min_wd, y.withdraw_r)
        series.loc[start] = min_wd
    return series

#    u = metrics.ulcer([y.portfolio_r for y in annual_data])
#    series.loc[start] = u

In [ ]:
vpw = run_one()    
capped = run_one(CappedVPW)
split = run_one(SplitVPW)
#plot.plot_n({'VPW' : vpw, 'Capped' : capped}, 'Retirement Year', 'Minimum Real Withdrawal')
plot.plot_n({'VPW' : vpw, 'SplitVPW' : split}, 'Retirement Year', 'Minimum Real Withdrawal')

In [ ]:
vpw.sort_values()

In [ ]:
YEAR = 1980
MARKET = market.Returns_US_1871()
years = 35
vpw = simulate.withdrawals(MARKET.iter_from(YEAR),
                                           years=years,
                                           harvesting=harvesting.N_60_RebalanceHarvesting,
                                           withdraw=withdrawal.VPW)
capped = simulate.withdrawals(MARKET.iter_from(YEAR),
                                           years=years,
                                           harvesting=harvesting.N_60_RebalanceHarvesting,
                                           withdraw=CappedVPW)

plot.plot_n({
    'VPW' : [n.withdraw_r for n in vpw],
    'Capped' : [n.withdraw_r for n in capped]
}, 'Year', 'Withdrawal $')

plot.plot_n({
    'VPW' : [n.portfolio_r for n in vpw],
    'Capped' : [n.portfolio_r for n in capped]
}, 'Year', 'Portfolio $')

In [ ]:
capped[34].portfolio_r

In [ ]:
def run_cew(withdrawal_strategy=withdrawal.VPW):
    MARKET = market.Returns_US_1871()

    years = 30
    end_year = 2018 - years + 1
    series = pandas.Series(index=numpy.arange(MARKET.start_year, end_year))

    for start in range(MARKET.start_year, end_year):
        annual_data = simulate.withdrawals(MARKET.iter_from(start),
                                           years=years,
                                           harvesting=harvesting.N_60_RebalanceHarvesting,
                                           withdraw=withdrawal_strategy)
        cew = metrics.cew([n.withdraw_r for n in annual_data])
        series.loc[start] = cew
    return series

#    u = metrics.ulcer([y.portfolio_r for y in annual_data])
#    series.loc[start] = u

In [ ]:
vpw = run_cew()
capped = run_cew(CappedVPW)
split = run_cew(SplitVPW)
#plot.plot_n({'VPW' : vpw, 'Capped' : capped}, 'Retirement Year', 'Certainty Equivalent Withdrawals')
plot.plot_n({'VPW' : vpw, 'SpiltVPW' : split}, 'Retirement Year', 'Certainty Equivalent Withdrawals')

In [ ]:
def compare(year, other_withdrawal_strategy):
    YEAR = year
    MARKET = market.Returns_US_1871()
    years = 35
    vpw = simulate.withdrawals(MARKET.iter_from(YEAR),
                                               years=years,
                                               harvesting=harvesting.N_60_RebalanceHarvesting,
                                               withdraw=withdrawal.VPW)
    other = simulate.withdrawals(MARKET.iter_from(YEAR),
                                               years=years,
                                               harvesting=harvesting.N_60_RebalanceHarvesting,
                                               withdraw=other_withdrawal_strategy)

    plot.plot_n({
        'VPW' : [n.withdraw_r for n in vpw],
        other_withdrawal_strategy.__name__ : [n.withdraw_r for n in other]
    }, 'Year', 'Withdrawal $ for %s retiree' % year)

    plot.plot_n({
        'VPW' : [n.portfolio_r for n in vpw],
        other_withdrawal_strategy.__name__ : [n.portfolio_r for n in other]
    }, 'Year', 'Portfolio $ for %s retiree' % year)
    
    return (vpw, other)

In [ ]:
results = compare(1906, SplitVPW)
def c(results):
    vpw, split = results
    vpw_wd = [n.withdraw_r for n in vpw]
    vpw_p = [n.portfolio_r for n in vpw]
    split_wd = [n.withdraw_r for n in split]
    split_p = [n.portfolio_r for n in split]
    
    diff = pandas.DataFrame(index=range(len(vpw_wd)), columns=['VPW', 'Split', 'Diff'])

    for i in range(len(vpw_wd)):
        diff.iloc[i] = (vpw_wd[i] / vpw_p[i], split_wd[i] / split_p[i], split_wd[i] - vpw_wd[i])
        
    #diff.to_csv('split-diff.csv')
    for i in range(10,20):
        print(split_p[i])
    
    print(diff.iloc[15])
    
c(results)

In [ ]:
def check90(year):
    YEAR = year
    MARKET = market.Returns_US_1871()
    years = 35
    r = simulate.withdrawals(MARKET.iter_from(YEAR),
                                               years=years,
                                               harvesting=harvesting.N_60_RebalanceHarvesting,
                                               withdraw=withdrawal.ConstantDollar)
    series = pandas.Series([n.portfolio_r for n in r])
    under = series[series < 900000]
    pct = len(under) / len(series)
    
    return pct

In [ ]:
pct_series = pandas.Series(index=range(1871, 2018-35+1))
for year in range(1871, 2018 - 35 + 1):
    pct_series.loc[year] = check90(year)

In [ ]:
pct_series.sort_values()